<a href="https://colab.research.google.com/github/LucyMariel/Lucy/blob/master/KERAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction to Keras**

Keras is a high-level, deep learning API developed by Google for implementing neural networks. It is written in Python and is used to make the implementation of neural networks easy. It also supports multiple backend neural network computation.

Keras was originally a library that appeared as a wrapper that made it easier to handle multiple frameworks such as Theano and TensorFLow, but later it was used as a high-level API for TensorFLow.

We will mainly use tf.keras, which is Keras included in TensorFLow.

<< What is a wrapper >>

Wrappers take advantage of the functionality of the original program to provide something easier to use. TensorFlow provides the ability to efficiently perform the calculations required for neural networks, but in the early days it was time-consuming to build and train models. That's why Keras, which wraps TensorFlow to make it easier to handle, has arrived.

<< What is a high-level API? >>

A high-level API is created so that functions can be easily handled in large units. As an antonym, there is a low-level API that allows you to tweak functions in small units, but it is difficult to handle.

In TensorFlow itself, high-level APIs are being enhanced to facilitate model construction and learning of neural networks. tf.Keras is one of them.

 Implementation of logistic regression

Let's create an AND gate by logistic regression as in the introduction to TensorFLow 2.

First, prepare the AND gate data.

In [1]:
import numpy as np
# ANDゲートの学習データを用意
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

Multiple description methods

Keras makes it easy to write neural networks. There are two ways to write it: Sequential model and Functional API. Let's look at each.

Sequential model

This is a description method using the Sequential class.

tf.keras.models.Sequential | TensorFlow

Define the model by passing an instance of the layer in a list to the constructor of the Sequential class. The layer classes are organized on the following pages.

Module: tf.keras.layers | TensorFlow

To make a logistic regression, we use the fully connected layer class, tf.keras.layers.Dense. Enter the number of output units, activation function, and number of input units as arguments.

In [2]:
import tensorflow as tf

model = tf.keras.Sequential([tf.keras.layers.Dense(1, activation = tf.nn.sigmoid, input_shape=(2,))])

The Dense class can also specify the weight initialization method, bias presence / absence, etc. with arguments.

tf.keras.layers.Dense | TensorFlow

You can check the structure of the created model with the summary method. The output shape and the number of parameters for each layer are also listed.

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 3         
                                                                 
Total params: 3 (12.00 Byte)
Trainable params: 3 (12.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Once you have written the structure, compile the model. Specify the loss function, optimization method, and evaluation function at compile time. The loss function is named as a string. Here, it is binary_crossentropy because it is a binary classification. For multi-valued classification it is categorical_crossentropy, for regression it is mean_squared_error.

In [4]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

AttributeError: module 'tensorflow._api.v2.train' has no attribute 'AdamOptimizer'

And we will learn. It is designed to use the fit method as well as scikit-learn. **verbose is a parameter of the learning process visualization method, and the default 1 is to display a progress bar that is updated on a batch-by-batch basis. If verbose is 0, it will not be displayed, and if it is 2, it will be displayed for each epoch**.

In [5]:
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

Although it is not prepared now, if there is verification data, it is possible to verify each epoch by giving it to the argument validation_data.

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1,
                    validation_data=(x_train, y_train))

Estimating uses the predict method as well as scikit-learn.

In [ ]:
y_pred_proba = model.predict(x_train)[:, 0]

# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)

print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)

The evaluate method is also useful if you don't need the results and just want to evaluate.

In [ ]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

**Another way to write a Sequential model**
In the Sequential model, it is also common to write using the add method instead of passing the layer class in the constructor.

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1, activation = tf.nn.sigmoid, input_shape=(2,)))

In the case of a two-layer neural network instead of logistic regression, it can be described as follows. It is not necessary to give input_shape for the second and subsequent layers. This is because tf.keras calculates it automatically.

In [ ]:
model = tf.keras.Sequential([
            tf.keras.layers.Dense(10, activation = tf.nn.relu, input_shape=(2,)),
            tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)])

If you use the add method, it will look like this:

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu, input_shape=(2,)))
model.add(tf.keras.layers.Dense(1, activation = tf.nn.sigmoid))

 Functional API
You can build a model with a higher degree of freedom by using the Functional API. Use the Model class instead of the Sequential class.

tf.keras.models.Model | TensorFlow

Describe the flow from input to output, and finally pass the input layer and the instance of the output layer to the Model class.

In [ ]:
input_data = tf.keras.layers.Input(shape=(2,)) # 入力層
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(input_data) # output層

model = tf.keras.Model(inputs=input_data, outputs=output)

After describing the model structure, it is exactly the same as the Sequential model.

In [ ]:
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1)

For multiple layers
A 4-layer neural network can be described as follows.

In [ ]:
input_data = tf.keras.layers.Input(shape=(2,))
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=input_data, outputs=output)

This description method can also express branching. The following is an example of branching into two in the third layer and joining in the next layer.
Link: tf.keras.layers.concatenate | TensorFlow

In [ ]:
input_data = tf.keras.layers.Input(shape=(2,))
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
y1 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
y2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
z = tf.keras.layers.concatenate([y1, y2])
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(z)
model = tf.keras.Model(inputs=input_data, outputs=output)

Keras as a wrapper
Keras as a wrapper also uses TensorFlow as the backend by default, so the basic usage is the same.

The document is also published in Japanese, so you can refer to it when using tf.keras. For example, the following page describes the two description methods.

Link: Sequential Model Guide-Keras Documentation

Linf: Functional API Guide-Keras Documentation

The loss functions that can be specified with the compile method are also grouped together.

Link: Loss function-Keras Documentation

The Sequential model can be written as: An example of logistic regression.

Most of the code below is essentially the same as tf.keras introduced above, but for example the activation function is passed as a separate class from the fully connected layer. Also, the optimization method part has changed from tf.train.AdamOptimizer to keras.optimizers.Adam. This is because tf.keras calls the optimization method class of TensorFlow itself, while Keras uses Keras's own optimization method class. I often see Keras code as a wrapper, so it's a good idea to get used to the slight differences.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(1, input_shape=(2,)))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1)

[Problem 1] Sharing and executing the official tutorial model
Please share and execute the official tutorial model of TensorFLow.

Link: models/tutorials at master · tensorflow/models

[Problem 2] (Advance assignment) Execute various methods
The GitHub repositories of TensorFLow and Google AI Research have a wide variety of code from classic models to the latest models. Select the one you are interested in and execute it.

Please note that these codes are not for beginners and may not be easy to execute, such as when you need to download a huge dataset. In that case, please do a code reading.


Lynk: models/research at master · tensorflow/models

Lynk: google-research/google-research: Google AI Research

Those with older update dates may have older versions of Python and TensorFlow and may be unwieldy. It is recommended to look at the newest ones first.

Rewriting to a different framework

We will change the code of TensorFLow that handles 4 types of data sets created with "Sprint 13 TensorFlow" to a different framework.

- Iris (binary classification only for Iris-versicolor and Iris-virginica)
- Iris (multi-value classification using all three objective variables)
- House Prices
- MNIST

Rewriting to Keras

For Keras, use the tf.keras module included in TensorFLow.

There are various types of writing in Keras such as Sequential model or Functional API, but this is not specified.



[Problem 3] Learning Iris (binary classification) with Keras

Rewrite the binary classification for the Iris dataset by TensorFlow to Keras.

[Problem 4] Learn Iris (multi-level classification) with Keras

Rewrite the ternary classification for Iris dataset by TensorFlow to Keras.

[Problem 5] Learning House Prices with Keras

Rewrite the regression on the House Prices data set by TensorFlow to Keras.

[Problem 6] Learning MNIST with Keras

Please rewrite the multi-valued classification of images by MNIST dataset by TensorFlow to Keras.

[Problem 7] (Advance assignment) Rewriting to PyTorch

Please rewrite 4 kinds of problems to PyTorch.

Link: pytorch:tutorials

[Problem 8] (Advance assignment) Comparison of frameworks

Please summarize the differences between the frameworks.

<Example of viewpoint>

- Calculation speed
- Number of lines of code and readability
- Functions provided